In [ ]:
import os

def send_linux_alert(message):
    os.system(f"notify-send \"{message}\"")

In [ ]:
import talib
import yfinance as yf
from datetime import datetime, timedelta





def check_stock_trend(day, start_ts,end_ts, ticker, period):
    """Checks for a trend in the stock price using candlestick patterns and technical indicators.

    Args:
        start_date: The start date of the period to check.
        end_date: The end date of the period to check.
        ticker: The stock ticker.
        period: The chart period (e.g., '1m' for 1-minute chart).

    Returns:
        The found trend (up or down) if a trend is detected, otherwise None.
    """
    next_day = day + timedelta(days=1)

    # Download historical data
    try:
        ticker_data = fetch_data(ticker, start=day.strftime("%Y-%m-%d"), end=next_day.strftime("%Y-%m-%d"), interval=period)
        if ticker_data is None:
            #print(ticker + " failed to download, skipping")
            return None,None
        df = ticker_data.between_time(start_time=start_ts, end_time=end_ts)
        vol = df['Volume'].sum()
        # Initialize trend flags
        higher_highs = True
        higher_lows = True

        # Iterate through the data to check for higher highs and higher lows
        for i in range(len(df) - 1):
            current_high = df['High'].iloc[-(i + 1)]
            previous_high = df['High'].iloc[-(i + 2)]
            current_low = df['Low'].iloc[-(i + 1)]
            previous_low = df['Low'].iloc[-(i + 2)]

            if current_high <= previous_high:
                higher_highs = False
            if current_low <= previous_low:
                higher_lows = False

        if higher_highs and higher_lows:
            return "up trend",vol
        elif not higher_highs and not higher_lows:
            return "down trend",vol
        else:
            return None, None
    except Exception as e:
        return f"Error occurred: {e}"
    # # Calculate candlestick pattern indicators
    # highs = limited_ts['High']
    # lows = limited_ts['Low']
    # higher_highs = talib.HT_TRENDLINE(highs)
    # print(higher_highs)
    # higher_lows = talib.HT_TRENDLINE(lows)

    # # Calculate MACD and RSI
    # macd, macdsignal, macdhist = talib.MACD(limited_ts['Close'])
    # rsi = talib.RSI(limited_ts['Close'], timeperiod=14)

    # # Determine trend based on multiple indicators
    # if (higher_highs > higher_lows) and (macd > macdsignal) and (rsi > 30):
    #     trend = 'up'
    # elif (higher_highs < higher_lows) and (macd < macdsignal) and (rsi < 70):
    #     trend = 'down'
    # else:
    #     trend = None

    # Send a system notification if a trend is found
    

    return trend

In [20]:
tickers =  [
        "AAPL",  # Apple Inc.
        "MSFT",  # Microsoft Corporation
        "AMZN",  # Amazon.com Inc.
        "GOOGL",  # Alphabet Inc. (Class A)
        "GOOG",  # Alphabet Inc. (Class C)
        "META",  # Meta Platforms Inc.
        "TSLA",  # Tesla Inc.
        "BRK.B",  # Berkshire Hathaway Inc. (Class B)
        "NVDA",  # NVIDIA Corporation
        "JPM",  # JPMorgan Chase & Co.
        "JNJ",  # Johnson & Johnson
        "V",  # Visa Inc.
        "PG",  # Procter & Gamble Co.
        "UNH",  # UnitedHealth Group Incorporated
        "HD",  # The Home Depot Inc.
        "MA",  # Mastercard Incorporated
        "DIS",  # The Walt Disney Company
        "PYPL",  # PayPal Holdings Inc.
        "NFLX",  # Netflix Inc.
        "INTC",  # Intel Corporation
        "VZ",  # Verizon Communications Inc.
        "KO",  # The Coca-Cola Company
        "PFE",  # Pfizer Inc.
        "MRK",  # Merck & Co. Inc.
        "CSCO",  # Cisco Systems Inc.
        "PEP",  # PepsiCo Inc.
        "ABT",  # Abbott Laboratories
        "COST",  # Costco Wholesale Corporation
        "CMCSA",  # Comcast Corporation
        "XOM",  # Exxon Mobil Corporation
        "BAC",  # Bank of America Corporation
        "WMT",  # Walmart Inc.
        "ADBE",  # Adobe Inc.
        "NKE",  # NIKE Inc.
        "T",  # AT&T Inc.
        "CRM",  # Salesforce.com Inc.
        "MCD",  # McDonald's Corporation
        "QCOM",  # Qualcomm Incorporated
        "MDT",  # Medtronic plc
        "LLY",  # Eli Lilly and Company
        "ORCL",  # Oracle Corporation
        "NEE",  # NextEra Energy Inc.
        "UPS",  # United Parcel Service Inc.
        "IBM",  # International Business Machines Corporation
        "TXN",  # Texas Instruments Incorporated
        "HON",  # Honeywell International Inc.
        "CVX",  # Chevron Corporation
        "BA",  # The Boeing Company
        "WFC",  # Wells Fargo & Company
    ]

In [ ]:
tickers=["NVDA"]

In [19]:
import yfinance as yf
import talib
import pandas as pd
from datetime import datetime, timedelta
import sys,os

def fetch_data(ticker, start, end, interval="1m"):
    ticker_data = yf.download(ticker, start, end, interval)
    if ticker_data.empty:
        print(ticker + " failed to download, skipping")
        return None
    else:
        print("data downloaded successfully for "+
        +ticker)
        return ticker_data

def check_stock_trend( start_ts, end_ts, ticker, period):
    """Checks for a trend in the stock price using candlestick patterns and technical indicators.

    Args:
        day (datetime): The day to check for trend.
        start_ts (str): Start time for slicing the data (e.g., '09:30').
        end_ts (str): End time for slicing the data (e.g., '16:00').
        ticker (str): The stock ticker symbol.
        period (str): The interval for the data (e.g., '1m' for 1-minute).

    Returns:
        tuple: (trend, volume) where trend is 'up trend', 'down trend', or None, and volume is the total volume.
    """
    #next_day = day + timedelta(days=1)

    try:
        # Download historical data
        #ticker_data = fetch_data(ticker, start=day.strftime("%Y-%m-%d"), end=next_day.strftime("%Y-%m-%d"), interval=period)
        ticker_data = fetch_data(ticker, start=start_ts, end=end_ts, interval=period)
        if ticker_data is None:
            print(ticker + " failed to download, skipping")
            return None,None

        # Slice data and avoid chained assignment warning
        df = ticker_data.copy()  # Work on a copy of the original DataFrame
        df = df.between_time(start_time=start_ts, end_time=end_ts)
        print(df)
        
        if df.empty:
            return None, None  # No data in the specified time range

        # Calculate technical indicators
        df['SMA'] = talib.SMA(df['Close'], timeperiod=20)
        df['EMA'] = talib.EMA(df['Close'], timeperiod=20)
        df['RSI'] = talib.RSI(df['Close'], timeperiod=14)
        df['MACD'], df['MACD Signal'], df['MACD Hist'] = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

        # Initialize trend flags
        higher_highs = True
        higher_lows = True

        # Iterate through the data to check for higher highs and higher lows
        for i in range(len(df) - 1):
            current_high = df['High'].iloc[-(i + 1)]
            previous_high = df['High'].iloc[-(i + 2)]
            current_low = df['Low'].iloc[-(i + 1)]
            previous_low = df['Low'].iloc[-(i + 2)]

            if current_high <= previous_high:
                higher_highs = False
            if current_low <= previous_low:
                higher_lows = False

        # Determine trend
        if higher_highs and higher_lows:
            return "up trend", df['Volume'].sum()
        elif not higher_highs and not higher_lows:
            return "down trend", df['Volume'].sum()
        else:
            return None, None

    except Exception as e:
        return None, f"Error occurred: {e}"


In [ ]:
from datetime import datetime
import pytz
# Define the timezone (Eastern Time for NVDA, since it's traded on NASDAQ)
tz = pytz.timezone('US/Eastern')

# Get the current time in Eastern Time
end_time = datetime.now(tz)
start_time = end_time - timedelta(minutes=15)
for ticker in tickers:
    trend, volume = check_stock_trend(start_ts=start_time,end_ts=end_time,ticker=ticker,period="1m")
    if trend :
            print(ticker+": "+trend+" . Volume: "+str(volume))
            if os.path.exists('/usr/bin/notify-send'):  # Check if notify-send is available
                os.system(f"notify-send \"The stock price of {ticker} is trending {trend}.\"")
            else:
                print("notify-send is not available on your system.")

In [29]:
def check_trend(ticker,start,end):
    import yfinance as yf
    from datetime import datetime, timedelta
    import pytz

    # Define the stock ticker
    ticker = ticker

    # start_time=datetime(year=2024,month=9,day=13,hour=9,minute=30)
    # end_time=datetime(year=2024,month=9,day=13,hour=9,minute=45)
    start_time=start
    end_time=end
    # Fetch 1-minute interval data for the last 15 minutes
    data = yf.download(ticker, start=start_time, end=end_time, interval='1m')

    # Convert the index to the appropriate timezone (if not already)
    #data.index = data.index.tz_localize('UTC').tz_convert(tz)

    highs = data['High']
    lows = data['Low']
    print(data)
    higher_highs = all(x < y for x, y in zip(highs, highs[1:]))
    higher_lows = all(x < y for x, y in zip(lows, lows[1:]))
    lower_highs = all(x > y for x, y in zip(highs, highs[1:]))
    lower_lows = all(x > y for x, y in zip(lows, lows[1:]))
    if higher_highs and higher_lows:
        trend = "Uptrend"
    elif lower_highs and lower_lows:
        trend = "Downtrend"
    else:
        trend = "No clear trend"
    if trend!="No clear trend":
        print(ticker + ": "+trend)
        print(data)

In [ ]:
start=datetime(year=2024,month=9,day=13,hour=17,minute=30)
end=datetime(year=2024,month=9,day=13,hour=17,minute=42)
for ticker in tickers:
    check_trend(ticker,start=start,end=end)

In [33]:
import yfinance as yf
import datetime
import pytz

def download_stock_data(ticker, minutes, timezone="Europe/Paris"):
    """Downloads 1-minute historical data for a given stock, considering a specified timezone.

    Args:
        ticker (str): The stock ticker symbol.
        minutes (int): The number of minutes of data to download.
        timezone (str): The timezone to use (default: Europe/Paris).

    Returns:
        pandas.DataFrame: The downloaded data.
    """

    # Create a timezone object
    tz = pytz.timezone(timezone)

    # Get the current time in the specified timezone
    now = datetime.datetime.now(tz)

    # Calculate the start time, ensuring it's not in the future
    start_time = now - datetime.timedelta(minutes=minutes)
    if start_time > now:
        start_time = now

    # Download the data, specifying the timezone
    data = yf.download(ticker, start=start_time, end=now, interval="1m", tz=timezone)

    return data

# Example usage:
ticker = "AAPL"  # Replace with your desired ticker
minutes = 60  # Replace with the desired number of minutes

data = download_stock_data(ticker, minutes)

print(data)

TypeError: download() got an unexpected keyword argument 'tz'